In [64]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.datasets import load_digits

In [65]:
digits = load_digits()
print(digits.data.shape)
print(digits.target.shape)
N = digits.data.shape[0]
d = digits.data.shape[1]
A = digits.data
y = 1.0*np.ones([A.shape[0],1])
for i in range(digits.target.shape[0]):
  y[i] = digits.target[i]


(1797, 64)
(1797,)


In [66]:
def evalf(x,l):
  assert type(x) is np.ndarray
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = 0.5 * (fval)**2+(l/2)*np.matmul(x.T,x)
  return (fval)

In [67]:
def evalg(x,l):
  assert type(x) is np.ndarray
  grad=(l*x+np.matmul(A.T, np.matmul(A, x) - y))
  return grad

In [68]:
def evalh(x,l):
  assert type(x) is np.ndarray
  return np.matmul(A.T,A)+l*np.identity(d)

In [69]:
def compute_B_k(s, y, B_k):
  I=np.identity(64)
  assert type(s) is np.ndarray
  assert type(y) is np.ndarray
  assert type(B_k) is np.ndarray
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [70]:
#line search type
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [71]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  mat = evalh(x)
  return np.linalg.inv(mat)

In [72]:
def compute_steplength_backtracking_scaled_direction(x, lam,gradf, alpha_start, rho, gamma, B_k):
  assert type(x) is np.ndarray
  assert type(gradf) is np.ndarray
  assert type(alpha_start) is float and alpha_start>=0.
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0.
  alpha = alpha_start
  p = - gradf
  D_k = B_k

  while evalf(x + alpha*np.matmul(D_k,p),lam) > evalf(x,lam) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha = alpha * rho
  return alpha

In [73]:
def minimizer_Newtonmethod(start_x,l, tol, line_search_type, *args):
  assert type(start_x) is np.ndarray
  assert type(tol) is float and tol>=0

  x = start_x
  g_x = evalg(x,l)
  h_x = evalh(x,l)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  while (np.linalg.norm(g_x) > tol):
    D_k = np.linalg.inv(evalh(x,l))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x,l, g_x, alpha_start, rho, gamma, D_k)
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.0
    else:
      raise ValueError('Line search type unknown!')

    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k = k + 1
    g_x = evalg(x,l)
  return x,  k

In [74]:
def minimizer_BFGS_scaling(start_x, l,tol, line_search_type, *args):
  assert type(start_x) is np.ndarray
  assert type(tol) is float and tol>=0

  x = start_x
  g_x = evalg(x,l)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  k = 0
  B_k = np.identity(d)
  x_old = x
  while (np.linalg.norm(g_x) > tol):
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old,l, g_x, alpha_start, rho, gamma, B_k)
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x)))
    g_x = evalg(x_new,l)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new,l), evalg(x_old,l))
    B_k = compute_B_k(s, y, B_k)
    x_old = x_new
    k = k + 1
  return x_new, k

In [75]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-3
l=0.001

In [76]:
my_start_x =np.zeros(d)
x, k= minimizer_Newtonmethod(my_start_x.reshape(d,1),l, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('for starting with',my_start_x,':\n minimizer:',x ,'\n no. of iterations:',k)

for starting with [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] :
 minimizer: [[ 0.00000000e+00]
 [ 9.69076881e-02]
 [-4.32192761e-03]
 [-7.75916338e-03]
 [ 7.49591986e-02]
 [ 1.13946582e-02]
 [-2.71293921e-02]
 [-7.34410663e-03]
 [ 9.98267906e-01]
 [-2.88089376e-02]
 [ 1.18688356e-01]
 [ 6.60922719e-02]
 [-5.57075914e-02]
 [-6.97056149e-02]
 [ 9.65844013e-02]
 [ 2.55196013e-01]
 [-7.29830665e-01]
 [ 2.42711745e-02]
 [ 7.73241543e-02]
 [-2.33008498e-02]
 [-5.64077618e-02]
 [ 5.72413971e-02]
 [-4.88675247e-02]
 [-2.62555997e-01]
 [-9.06071168e-01]
 [-1.49769638e-01]
 [ 5.64022789e-02]
 [ 8.96665318e-02]
 [ 8.39315937e-02]
 [ 9.85410046e-02]
 [ 1.69269848e-03]
 [-2.96649687e+00]
 [ 0.00000000e+00]
 [-1.54361470e-01]
 [-9.32404639e-03]
 [ 1.39497978e-01]
 [-3.69237437e-02]
 [ 5.46111772e-02]
 [-9.20425585e-03]
 [ 0.00000000e+00]
 [ 1.0332650

Regularized Ordinary Least Squares Linear Regression (OLSLR) helps us find the minimum point. When we use direct OLSLR, it often fails to reach the minimum because the mathematical matrix we use becomes problematic and cannot be flipped around properly. But with regularized OLSLR, we fix this issue. We do this by tweaking the matrix a bit, making sure it behaves nicely. This tweak allows us to find the minimum point successfully.

**2.**

In [77]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
l=0.001

In [78]:
my_start_x =np.zeros(d)
x, k = minimizer_BFGS_scaling(my_start_x.reshape((d,1)),l, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('for starting with',i,':\n minimizer:',x ,'\n no. of iterations:',k)

for starting with 1796 :
 minimizer: [[ 0.00000000e+00]
 [ 9.69076882e-02]
 [-4.32192766e-03]
 [-7.75916335e-03]
 [ 7.49591987e-02]
 [ 1.13946582e-02]
 [-2.71293921e-02]
 [-7.34410665e-03]
 [ 9.98267907e-01]
 [-2.88089376e-02]
 [ 1.18688356e-01]
 [ 6.60922718e-02]
 [-5.57075914e-02]
 [-6.97056149e-02]
 [ 9.65844014e-02]
 [ 2.55196013e-01]
 [-7.29830666e-01]
 [ 2.42711745e-02]
 [ 7.73241544e-02]
 [-2.33008497e-02]
 [-5.64077619e-02]
 [ 5.72413971e-02]
 [-4.88675248e-02]
 [-2.62555997e-01]
 [-9.06071169e-01]
 [-1.49769638e-01]
 [ 5.64022790e-02]
 [ 8.96665319e-02]
 [ 8.39315939e-02]
 [ 9.85410047e-02]
 [ 1.69269847e-03]
 [-2.96649688e+00]
 [ 0.00000000e+00]
 [-1.54361470e-01]
 [-9.32404637e-03]
 [ 1.39497978e-01]
 [-3.69237437e-02]
 [ 5.46111772e-02]
 [-9.20425583e-03]
 [ 0.00000000e+00]
 [ 1.03326506e-01]
 [ 1.23983484e-01]
 [-1.37635229e-02]
 [ 5.40029024e-03]
 [ 1.31185700e-01]
 [ 5.49577815e-02]
 [ 2.24935899e-02]
 [ 7.48046259e-03]
 [ 6.17507773e-01]
 [ 2.44100619e-02]
 [ 1.42356430

In [79]:
def evalf(x):
  assert type(x) is np.ndarray
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = 0.5 * (fval)**2
  return (fval)

In [80]:
def evalg(x):
  assert type(x) is np.ndarray
  return np.matmul(A.T, np.matmul(A, x) - y)

In [81]:
def evalh(x):
  assert type(x) is np.ndarray
  return np.matmul(A.T,A)

In [82]:
def compute_B_k(s, y, B_k):
  assert type(s) is np.ndarray
  assert type(y) is np.ndarray
  assert type(B_k) is np.ndarray
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  I = np.identity(64)
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [83]:
#line search type
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [84]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  mat = evalh(x)
  return np.linalg.inv(mat)

In [85]:
def compute_steplength_backtracking_scaled_direction(x, gradf, alpha_start, rho, gamma, B_k): #add appropriate arguments to the function
  assert type(x) is np.ndarray
  assert type(gradf) is np.ndarray
  assert type(alpha_start) is float and alpha_start>=0.
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0.
  n = len(x)
  alpha = alpha_start
  p = - gradf
  D_k = B_k
  while evalf(x + alpha*np.matmul(D_k,p)) > evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha = alpha * rho
  return alpha

In [86]:
def minimizer_Newtonmethod(start_x, tol, line_search_type, *args):
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments
  assert type(tol) is float and tol>=0

  x = start_x
  g_x = evalg(x)
  h_x = evalh(x)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  step_length = 1.
  while (np.linalg.norm(g_x) > tol):
    D_k = np.linalg.inv(evalh(x))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x, alpha_start, rho, gamma, D_k)
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.
    else:
      raise ValueError('Line search type unknown,check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x))) #update x = x - step_length*g_x
    k += 1
    g_x = evalg(x)
  return x,  k

In [87]:
def minimizer_BFGS_scaling(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments
  assert type(tol) is float and tol>=0

  x = start_x
  g_x = evalg(x)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  k = 0
  B_k = np.identity(d)
  x_old = x
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old, g_x, alpha_start, rho, gamma, B_k)
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.
    else:
      raise ValueError('Line search type unknown,check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x)))
    g_x = evalg(x_new)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new), evalg(x_old))
    B_k = compute_B_k(s, y, B_k)
    x_old = x_new
    k += 1
  return x_new, k

In [88]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5

In [89]:
my_start_x =np.zeros(d)
x, k= minimizer_Newtonmethod(my_start_x.reshape(d,1), my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('Values of x star:\n',x,'\nk=', k)

LinAlgError: Singular matrix

we got singular matrix which inverse is not possible.So that the dierct method OLSLR does not work.

In [90]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
my_start_x =np.zeros(d)
x, k= minimizer_BFGS_scaling(my_start_x.reshape(d,1), my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
print('Values of x*:\n',x,'\nk=', k)

Values of x*:
 [[ 0.00000000e+00]
 [ 9.69033568e-02]
 [-4.32277232e-03]
 [-7.76028319e-03]
 [ 7.49594380e-02]
 [ 1.13947198e-02]
 [-2.71328245e-02]
 [-7.33176333e-03]
 [ 9.98337968e-01]
 [-2.88095538e-02]
 [ 1.18688288e-01]
 [ 6.60916265e-02]
 [-5.57069862e-02]
 [-6.97063705e-02]
 [ 9.65876439e-02]
 [ 2.55182251e-01]
 [-7.29828608e-01]
 [ 2.42709916e-02]
 [ 7.73249597e-02]
 [-2.33000278e-02]
 [-5.64086144e-02]
 [ 5.72426822e-02]
 [-4.88717684e-02]
 [-2.62467763e-01]
 [-9.06562829e-01]
 [-1.49767791e-01]
 [ 5.64019538e-02]
 [ 8.96663590e-02]
 [ 8.39318159e-02]
 [ 9.85411936e-02]
 [ 1.69317614e-03]
 [-2.96805758e+00]
 [ 0.00000000e+00]
 [-1.54362338e-01]
 [-9.32361205e-03]
 [ 1.39497628e-01]
 [-3.69234835e-02]
 [ 5.46111776e-02]
 [-9.20505070e-03]
 [ 0.00000000e+00]
 [ 1.03279535e-01]
 [ 1.23983258e-01]
 [-1.37639605e-02]
 [ 5.40087816e-03]
 [ 1.31185107e-01]
 [ 5.49570758e-02]
 [ 2.24938237e-02]
 [ 7.47977909e-03]
 [ 6.17755030e-01]
 [ 2.44122357e-02]
 [ 1.42333037e-03]
 [-6.21110760e-0

**2)** \
In our Regularized OLSLR approach, we set λ to 0.1. This resulted in 114 iterations to converge. Surprisingly, when we switched to the Direct OLSLR method, we found it also took 114 iterations to converge. It's possible that the only difference between the two methods lies in the λ value: 0.1 in Regularization OLSLR versus 0 (or perhaps a very small value like 0.001) in Direct OLSLR. So, this small adjustment in the λ value likely explains why we observed the same number of iterations in both methods.

